**9.	Which player has won the most tournaments in a single month?**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

Reading requierd tables

In [2]:
MatchTournamentInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchTournamentInfo.csv')
MatchEventInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchEventInfo.csv')
MatchHomeTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchHomeTeamInfo.csv')
MatchAwayTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchAwayTeamInfo.csv')

Drop duplicated data

Creating a new table which contains match id, winer name and id and tournament name

In [9]:
merged_HW = MatchHomeTeamInfo[['match_id' ,
                                  'player_id']].merge(MatchAwayTeamInfo[['match_id' ,
                                                                         'player_id']],
                                                                         on ='match_id').drop_duplicates(subset='match_id')
merged_EHW = merged_HW[['match_id' ,
                        'player_id_x' , 'player_id_y']].merge(MatchEventInfo[['match_id',
                                                                              'winner_code',
                                                                              'start_datetime']],
                                                                               on ='match_id')
merged_TEHW = merged_EHW[['match_id' ,
                        'player_id_x',
                        'player_id_y',
                        'winner_code',
                        'start_datetime']].merge(MatchTournamentInfo[['match_id',
                                                                      'tournament_name',
                                                                      'tournament_id']],
                                                                       on ='match_id')
merged_TEHW = merged_TEHW.dropna()
merged_TEHW['winner_id'] = merged_TEHW.apply( lambda x : x['player_id_x']  
                                             if x['winner_code'] == 1 
                                             else x['player_id_y'] , axis = 1)


In [12]:
merged_TEHW.head()

,match_id,player_id_x,player_id_y,winner_code,start_datetime,tournament_name,tournament_id,winner_id
0,12260075,14486,254742,2.0,1714499700,"Madrid, Spain",129164,254742
1,12260076,206570,90080,1.0,1714480500,"Madrid, Spain",129164,206570
2,12260077,46391,275923,2.0,1714484700,"Madrid, Spain",129164,275923
3,12260078,106755,275923,1.0,1714564800,"Madrid, Spain",129164,106755
4,12260078,106755,275923,1.0,1714564800,"Madrid, Spain",129164,106755


determinig the last match and its winner according to start datetime

In [18]:
player_tounament = merged_TEHW.groupby('tournament_id',
                                        as_index=False).agg({'start_datetime': 'max' ,
                                                             'winner_id' :'last'})
player_tounament.head()

,tournament_id,start_datetime,winner_id
0,129164,1714929300,106755
1,129201,1714842900,228272
2,129510,1714891500,275471
3,129514,1714923000,48225
4,129518,1714922100,102151


Creating a data frame to show the number of won tournoments for each player

In [ ]:
player = pd.concat ([MatchHomeTeamInfo[['player_id', 'name']] , MatchAwayTeamInfo[['player_id', 'name']]]).drop_duplicates(subset = 'player_id')
player['no_tour'] = 0
for no in player['player_id']:
    player['no_tour'].loc[player['player_id'] == no] = len(player_tounament['tournament_id'].loc[player_tounament['winner_id'] == no].unique())

In [21]:
player.head()

,player_id,name,no_tour
0,14486,Nadal R.,0
1,206570,Sinner J.,0
2,46391,Struff J.,0
3,106755,Rublev A.,1
4,221012,Cerundolo F.,0


**Reporting the best player with the most wins**

In [22]:
best_no_tour = player['no_tour'].max()
best_winer_id = player['player_id'].loc[player['no_tour'] == player['no_tour'].max()].values[0]
best_winer_name = player['name'].loc[player['no_tour'] == player['no_tour'].max()].values[0]
print (f'Player {best_winer_name} with {best_winer_id} ID has won {best_no_tour} tournomernt which is most tournaments in a single month')


Player Świątek I. with 228272 ID has won 3 tournomernt which is most tournaments in a single month
